In [1]:
from bs4 import BeautifulSoup
import re
import requests
import pandas as pd

In [2]:
entry_urls = []

for i in range(0, 1000):
    entry_urls.append("https://rip.ie/death-notice/s/all?page=" + str(i) + "start=2015-01-01+00%3A00%3A00&end=today&sortField=a.createdAtCastToDate&sortDir=DESC&view=boxes")

"""
entry_urls = [
    "https://rip.ie/death-notice/s/all?page=1&start=2000-01-01+00%3A00%3A00&end=today&sortField=a.createdAtCastToDate&sortDir=DESC&view=boxes",
    "https://rip.ie/death-notice/s/all?page=2&start=2000-01-01+00%3A00%3A00&end=today&sortField=a.createdAtCastToDate&sortDir=DESC&view=boxes",
    "https://rip.ie/death-notice/s/all?page=3&start=2000-01-01+00%3A00%3A00&end=today&sortField=a.createdAtCastToDate&sortDir=DESC&view=boxes"
    # Add more entry URLs here
]
"""

'\nentry_urls = [\n    "https://rip.ie/death-notice/s/all?page=1&start=2000-01-01+00%3A00%3A00&end=today&sortField=a.createdAtCastToDate&sortDir=DESC&view=boxes",\n    "https://rip.ie/death-notice/s/all?page=2&start=2000-01-01+00%3A00%3A00&end=today&sortField=a.createdAtCastToDate&sortDir=DESC&view=boxes",\n    "https://rip.ie/death-notice/s/all?page=3&start=2000-01-01+00%3A00%3A00&end=today&sortField=a.createdAtCastToDate&sortDir=DESC&view=boxes"\n    # Add more entry URLs here\n]\n'

In [3]:
def scrape(html):
    soup = BeautifulSoup(html, 'html.parser')
    return soup

In [4]:
def scrape_url(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.text
        else:
            print(f"Failed to retrieve data from {url}. Status code: {response.status_code}")
            return None
    except Exception as e:
        print(f"Error while processing {url}: {str(e)}")
        return None

In [5]:
def get_url(soup):
    url_list = []
    anchors = soup.find_all('a', class_='showdown-dn-link')
    urls = [anchor['href'] for anchor in anchors]
    for url in urls:
        url_list.append("https://rip.ie" + url)
    return url_list

In [6]:
def get_page_data(entry_urls):
    data_list = []
    
    url_pattern = r'https://rip.ie/death-notice/([^/]+)-([^/]+)-([^/]+)(?:-([^/]+))?-(\d+)'
    
    for entry_url in entry_urls:
        html_data = scrape_url(entry_url)
        
        if html_data:
            url_list = get_url(scrape(html_data))
            
            for u in url_list:
                match = re.match(url_pattern, u)

                if match:
                    name = match.group(1)
                    city = match.group(2)
                    location = match.group(3)
                    identity = match.group(5)

                    data_list.append({
                                    "Name": name,
                                    "ID": identity,
                                    "City": city,
                                    "Location": location
                                })

                else:
                    print("Pattern did not match.")
                    
    df = pd.DataFrame(data_list)
    return df

In [7]:
result_df = get_page_data(entry_urls)
result_df

,Name,ID,City,Location
0,thomas-carroll,532269,louth,castlebellingham
1,wayne-lynch,532267,louth,castlebellingham
2,wayne-lynch,532267,monaghan,carrickmacross
3,ann-lynch,532266,dublin,finglas
4,jackie-john-wade,532265,dublin,beaumont
...,...,...,...,...
39995,elizabeth-elsie-cuffe,532245,galway,salthill
39996,elizabeth-elsie-cuffe,532245,westmeath,athlone
39997,bridget-mccann,532244,dublin,kinsealy
39998,pat-patrick-halpin,532243,dublin,swords
